In [7]:


def ons_run(fid, data, eta, beta, delta, tc):
    n, m = data.shape #n:rows, m:cols #n:507 m:30
    cum_ret = 1
    cumprod_ret = np.ones((n, 1), float) #507,1
    
    daily_ret = np.ones((n, 1), float) #507,1
    
    day_weight = np.ones((m, 1), float)/m  #30,1
    day_weight_o = np.zeros((m, 1), float) # 30,1
    daily_portfolio = np.zeros((n,m)) #507,30
    
    # print file head
    print(fid, '-------------------------------------\n')
    print(fid, 'Parameters [eta:%f, beta:%f, delta:%f, tc:%f]\n' %( eta, beta, delta, tc))
    print(fid, 'day\t Daily Return\t Total return\n')

    print(1, '-------------------------------------\n')
    
    for t in range(n):
        '''start from 0'''
        if t>=1:
            day_weight[:,t] = ons_kernel(data[0:t-2,:], day_weight[:,0:t-2], eta, beta, delta)
        
        #Normalize the constraint, always useless
        day_weight = np.divide(day_weight, np.sum(day_weight, axis=0)) # elementwise divide and column-sum
        daily_portfolio[t,:] = day_weight.conj().T
        
        # Cal t's return and total return
        daily_ret[t, 0] = np.dot(data[t,:],day_weight)*(1-tc/2*np.sum(np.absolute(day_weight-day_weight_o) ))
        cum_ret = cum_ret * daily_ret[t,0]
        cumprod_ret[t, 0] = cum_ret
        
        #Adjust weight[t, :] for the transaction cost issue
        day_weight_o = np.divide(day_weight[:,t] * data[t, :].conj().T , daily_ret[t,0])
        
        
    print('ONS(eta:%.2f, beta:%.2f, delta:%.2f, tc:%.4f), Final return: %.2f\n' % (eta, beta, delta, tc, cum_ret))
    print('-------------------------------------\n')
    print(1, 'ONS(eta:%.2f, beta:%.2f, delta:%.2f, tc:%.4f), Final return: %.2f\n' % (eta, beta, delta, tc, cum_ret))
    print('-------------------------------------\n')
    
def ons_kernel(data, weight_o, eta, beta, delta):
    weight = ons_expert(data, weight_o, eta, beta, delta)
    return weight

from cvxopt import solvers, matrix

def ons_expert(data, weight_o, eta, beta, delta):
    T, N = data.shape
    weight = weight_o
    print(weight.shape)
    
    #Compute b(t-1) and A(t-1)
    At = np.zeros(N)
    bt = np.zeros((N,1)) #Nx1 zeros
    grad = np.zeros((T,1))
    hessian = np.zeros(T)
    
    for i in range(T):
        grad = (np.dot(np.divide(data[i,:].T ,data[i,:] ), weight_o[:,i]))
        heissian = np.dot(grad,grad.conj().T)*(-1)
        At += (-1) * heissian
        bt += grad
        
    # Compute p(t)
    At += np.eye((N,N))
    bt = np.dot(bt ,(1+1/beta))
    
    qt = np.dot(np.linalg.solve(At,bt),delta) #linalg.solve(a,b) if a is square linalg.lstsq(a, b) otherwise
    
    #argmin(q-p)' * A(t-1)*(q-p)
    
    H = matrix(At * 2)
    f=matrix()
    A=np.array()
    b=np.array()
    Aeq = np.ones((N,1)).conj().T
    beq = np.ones((N,1)).conj().T * qt - 1
    lb = qt - np.ones((N,1))
    up = qt
    
    
    weight = solvers.qp(H, f, A, b, Aeq, beq)
    weight = qt - weight
    weight = (1-eta) * weight + eta * np.ones((N,1))/N
    
    return weight
    
    
# test ONS
'''
in OLPS default setting
learning rate =0
beta = 1
delta=0.125
transaction cost =0

final return=1.53
'''

import numpy as np
import scipy.io as sio
djia = sio.loadmat('djia.mat') # load a test input
print(djia['data'].shape)
ons_run(1, djia['data'], 0, 1, 0.125, 0.5)

(507, 30)
1 -------------------------------------

1 Parameters [eta:0.000000, beta:1.000000, delta:0.125000, tc:0.500000]

1 day	 Daily Return	 Total return

1 -------------------------------------

(30, 0)


IndexError: index -1 is out of bounds for axis 1 with size 0